In [1]:
from numba import jit
import numpy as np
from numba import jit, int32, float32, types, typed
from numba.typed import List
from numba.experimental import jitclass
import matplotlib.pyplot as plt

In [23]:
gyro_radius = 3.086*10**17 # 10 PeV proton in 1 mucro Gauss B field
steps_per_gyration = 50
c = 3*10**8

In [61]:
@jit(nopython=True, fastmath=True)
def propagate(particle_id, spheres):
    pos = [0.0, 0.0, 0.0]
    phi = 0.0
    particle_info = []
    direction = [1,1,1]
    isotrop = False
    distance = 0.0
    
    for i in range(1,10**5): 
        pos_prev = pos
        
        ### change direction #############################################################
        if np.random.random() < 0.0002:
            direction[0] = -1*direction[0]
        if np.random.random() < 0.003:
            direction[1] = -1*direction[1]
        if np.random.random() < 0.003:
            direction[2] = -1*direction[2]
            
        ### move in updated direction ####################################################
        if isotrop:
            normalize = (direction[0]**2+direction[1]**2+direction[2]**2)**0.5
            time = i # ToDo change this (need to incooperate steplength)
            for j in range(3):
                pos[j] = pos[j] + direction[j]/normalize
        else:
            phi_old = phi
            phi = phi_old + 2 * np.pi / steps_per_gyration
            ### move in phi direction 
            ### distance_travelled = 2*pi*gyro_radius / steps_per_gyration (only for small phi)
            delta_x = gyro_radius * (np.cos(phi) - np.cos(phi_old)) * direction[0]
            delta_y = gyro_radius * (np.sin(phi) - np.sin(phi_old)) * direction[0]
            ### move in rho direction
            ### distance_travelled = 2*pi*gyro_radius / steps_per_gyration
            delta_x = delta_x + np.cos(phi) * gyro_radius * direction[1] * 2 * np.pi / steps_per_gyration
            delta_y = delta_y + np.sin(phi) * gyro_radius * direction[1] * 2 * np.pi / steps_per_gyration
            ### move in z direction as much as in the x or y-direction
            delta_z = direction[2] * ((delta_x**2+delta_y**2)/2)**0.5
            ### move !!!
            pos[0] = pos[0] + delta_x
            pos[1] = pos[1] + delta_y
            pos[2] = pos[2] + delta_z
            ### distance_travelled
            distance_in_step = (delta_x**2+delta_y**2+delta_z**2)**0.5
            distance = distance + distance_in_step

        
        ### observer ####################################################################
        r2_prev = pos_prev[0]**2+pos_prev[1]**2+pos_prev[2]**2
        r2 = pos[0]**2+pos[1]**2+pos[2]**2
        if i<1000 or i%500 == 0:
            particle_info.append([particle_id, i, distance, pos[0], pos[1], pos[2], -1.0])
        for r2_sphere in spheres:
            if r2_prev > r2_sphere and r2 <= r2_sphere or r2_prev < r2_sphere and r2 >= r2_sphere:
                particle_info.append([particle_id, i, distance, pos[0], pos[1], pos[2], r2_sphere*1.0])
        
    return particle_info

In [62]:
@jit(nopython=True)
def many_particles(observer_spheres):
    data = [[0.0, 0.0, 0.0, 0.0, -1.0]]
    for i in range(10**2):
        particle_id = i
        data_new = propagate(particle_id, observer_spheres)
        data = data + data_new

    print('finished')
    return data

In [ ]:
observer_spheres = np.array([-1.0]) 
%time data = many_particles(observer_spheres)

In [ ]:
import pandas as pd
df = pd.DataFrame(data[1:])
df.columns = ['id', 'i', 'd', 'x', 'y', 'z', 'radius']
df

In [ ]:
# remove duplicated elements from list of times 
steps = []
[steps.append(i) for i in df['i'] if i not in steps]
kappa_perp = []
kappa_para = []
for i in steps:
    df_i = df.loc[df['i'] == i] # all particles per step
    kappa_perp.append(np.mean(np.array((df_i['x'].values**2+df_i['y'].values**2))/(4*np.array(df_i['d'].values))*c))
    kappa_para.append(np.mean(np.array(df_i['z'].values**2)/(2*np.array(df_i['d'].values))*c))

In [ ]:
from modules.Plotter import Plotter
import matplotlib.pyplot as plt
### each step is one second -> times = steps
plt.plot(np.array(steps)/steps_per_gyration, np.array(kappa_perp)*10**4, label='$\kappa_\perp$')
plt.plot(np.array(steps)/steps_per_gyration, np.array(kappa_para)*10**4, label='$\kappa_\parallel$')
plt.xlabel('distance [gyroorbit]')
plt.ylabel('$\kappa$ [cm$^2$/s]')
plt.legend()
plt.loglog()
plt.show()

In [235]:
print(np.log10(np.mean(kappa_para[-10:]))-np.log10(np.mean(kappa_perp[-10:])))

4.080085153263209


### Lessons learned

The parameter steps_per_gyration moves the first minimum linearly, which is expected -> increasing by 10 moves minima by factor 10 to right